# Building an intelligent LLM Routing System using Strands Agent SDK

This notebook demonstrates how to build an LLM workflow using [Strands Agent](https://github.com/strands-agents) for intelligent routing. 

## Use Case
Sales and marketing teams face diverse processing needs—from creative content generation to data-driven analysis to customer communications. Each of these tasks benefits from models with different strengths. Rather than using a one-size-fits-all approach, our routing system will analyze incoming requests, categorize them, and direct them to specialized models that excel at specific tasks.

Using Strands Agent, we'll be enable an LLM workflow that perform the following:

- Classify incoming requests by type and intent
- Create a decision framework that maps request categories to appropriate models
- Implement routing logic that directs requests to the optimal model

The following diagram depicts the overall architecture and the worklfow:

<img src="../../imgs/lab3-prompt-routing-architecture.png" width="800"> 

## Environment Setup

Installing Strands Agents, loading environment variables & initiate Strands Agent libraries

In [1]:
%pip install strands-agents strands-agents-tools -Uq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from strands import Agent
import boto3

# Set up AWS session
session = boto3.session.Session()
region = session.region_name
print(f"Using AWS region: {region}")

# Define model IDs for our routing system
classifier_model = "us.amazon.nova-micro-v1:0"  # For classification
strategy_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"  # For strategy tasks
social_model = "us.amazon.nova-lite-v1:0" # For social media tasks

Using AWS region: us-east-1


Setting up logger

In [3]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

## Agents creation
The following code block creates three specialized agents
- Classifier agent to classify the marketing requests
- Strategy agent to provide comprehensive strategic response to the requests
- Social Media agent to create social media posts with hashtags to the requests

In [4]:
# Create agents for different tasks
classifier_agent = Agent(
    model=classifier_model,
    system_prompt="""You are a specialized routing assistant for a Media & Entertainment marketing team.
    
Your job is to classify incoming marketing requests into one of these categories:

1. STRATEGY - Questions about developing marketing strategies, campaign planning, audience targeting, 
   marketing analytics, ROI assessment, or long-term marketing plans
   
2. SOCIAL_MEDIA - Requests for creating social media posts, captions, hashtags, or content for specific platforms

Analyze the request carefully and classify it as either "strategy" or "social_media".
Respond with ONLY the classification word, either "strategy" or "social_media".
"""
)

strategy_agent = Agent(
    model=strategy_model,
    system_prompt="""You are a marketing strategy expert for Media & Entertainment companies.
    
When responding to requests:
- Develop comprehensive strategic responses with actionable insights
- Include audience targeting recommendations
- Provide campaign planning frameworks
- Suggest ROI measurement approaches
- Consider industry trends and competitive landscape
- Structure your responses with clear sections and bullet points
"""
)

social_media_agent = Agent(
    model=social_model,  # Reusing the classifier model for social media tasks
    system_prompt="""You are a social media expert for Media & Entertainment companies.
    
When responding to requests:
- Create engaging social media content tailored to the platform
- Include creative captions that grab attention
- Suggest relevant hashtags to increase visibility
- Provide platform-specific tips for optimal engagement
- Consider current trends and best practices
- Format your content for easy copy-and-paste use
"""
)

The below code will route the user query to the appropriate agent based on the response from classifier agent

In [5]:
def route_request(user_query):
    """Route the user query to the appropriate agent based on classification"""
    
    print(f"📝 User Query: {user_query}\n")
    
    # Step 1: Classify the request
    print("🔍 Classifying request...")
    classification = classifier_agent(f"Classify this request: '{user_query}'")
    response_text = str(classification.message).lower()
    
    # Determine category
    if "strategy" in response_text:
        category = "strategy"
    else:
        category = "social_media"
        
    print(f"✅ Request classified as: {category.upper()}\n")
    
    # Step 2: Route to the appropriate agent
    if category == "strategy":
        print("🧠 Routing to Strategy Agent (Claude 3.5 Sonnet)...\n")
        response = strategy_agent(user_query)
    else:  # social_media
        print("📱 Routing to Social Media Agent (Claude 3 Haiku)...\n")
        response = social_media_agent(user_query)
    
    response_text = str(response.message)
    
    print("\n" + "-"*50 + "\n")
    print(f"🤖 Response:\n\n{response_text}")
    print("\n" + "-"*50)
    
    return {"category": category, "response": response_text}

### Validation

In [6]:
# Test with a strategy request
strategy_query = "We need to analyze the ROI of our summer blockbuster campaign across different marketing channels. Can you develop a framework for measuring effectiveness?"
route_request(strategy_query)

INFO | strands.telemetry.metrics | Creating Strands MetricsClient


📝 User Query: We need to analyze the ROI of our summer blockbuster campaign across different marketing channels. Can you develop a framework for measuring effectiveness?

🔍 Classifying request...
strategy✅ Request classified as: STRATEGY

🧠 Routing to Strategy Agent (Claude 3.5 Sonnet)...

# Blockbuster Movie Marketing ROI Framework

## Campaign Effectiveness Measurement Structure

### Pre-Campaign Baseline Metrics
- **Box Office Projections:** Establish baseline expectations based on genre, cast appeal, and similar releases
- **Audience Awareness:** Conduct pre-campaign surveys measuring title recognition and intent-to-view
- **Social Media Benchmarks:** Document follower counts, engagement rates, and sentiment across platforms
- **Search Volume Analysis:** Record pre-campaign search trends for film title and related keywords

### Channel-Specific ROI Measurement

**Traditional Media**
- **TV Advertising:**
  * Cost per impression (CPM) compared to audience reach
  * Correlation betwe

{'category': 'strategy',
 'response': "{'role': 'assistant', 'content': [{'text': '# Blockbuster Movie Marketing ROI Framework\\n\\n## Campaign Effectiveness Measurement Structure\\n\\n### Pre-Campaign Baseline Metrics\\n- **Box Office Projections:** Establish baseline expectations based on genre, cast appeal, and similar releases\\n- **Audience Awareness:** Conduct pre-campaign surveys measuring title recognition and intent-to-view\\n- **Social Media Benchmarks:** Document follower counts, engagement rates, and sentiment across platforms\\n- **Search Volume Analysis:** Record pre-campaign search trends for film title and related keywords\\n\\n### Channel-Specific ROI Measurement\\n\\n**Traditional Media**\\n- **TV Advertising:**\\n  * Cost per impression (CPM) compared to audience reach\\n  * Correlation between ad spot timing and ticket pre-sales\\n  * Attribution modeling using unique landing page URLs or promo codes\\n  \\n- **OOH & Print:**\\n  * Geospatial analysis comparing loca

In [7]:
# Test with a social media request
social_media_query = "We need 5 Instagram post ideas for the premiere of our new sci-fi series 'Nexus Point.' Please include caption text and hashtag suggestions."
route_request(social_media_query)

📝 User Query: We need 5 Instagram post ideas for the premiere of our new sci-fi series 'Nexus Point.' Please include caption text and hashtag suggestions.

🔍 Classifying request...
social_media✅ Request classified as: SOCIAL_MEDIA

📱 Routing to Social Media Agent (Claude 3 Haiku)...

Absolutely! Here are five Instagram post ideas for the premiere of your new sci-fi series 'Nexus Point':

### Post 1: Teaser Trailer Reveal

**Image/Video:** A dynamic, eye-catching frame from the trailer with a futuristic cityscape in the background.

**Caption:**
"🌌 Dive into the unknown with 'Nexus Point'! 🚀 Our new sci-fi series premieres [Date]. Are you ready to explore the cosmos? 🌠 #NexusPoint #SciFiAdventure #PremiereAlert"

**Hashtags:** #NexusPoint #SciFiSeries #PremiereNight #FuturisticAdventure #SpaceExploration

### Post 2: Character Introduction

**Image/Video:** A high-quality image of a main character in a dramatic pose.

**Caption:**
"Meet [Character Name], the hero of 'Nexus Point'. 🌟 Wil

{'category': 'social_media',
 'response': '{\'role\': \'assistant\', \'content\': [{\'text\': \'Absolutely! Here are five Instagram post ideas for the premiere of your new sci-fi series \\\'Nexus Point\\\':\\n\\n### Post 1: Teaser Trailer Reveal\\n\\n**Image/Video:** A dynamic, eye-catching frame from the trailer with a futuristic cityscape in the background.\\n\\n**Caption:**\\n"🌌 Dive into the unknown with \\\'Nexus Point\\\'! 🚀 Our new sci-fi series premieres [Date]. Are you ready to explore the cosmos? 🌠 #NexusPoint #SciFiAdventure #PremiereAlert"\\n\\n**Hashtags:** #NexusPoint #SciFiSeries #PremiereNight #FuturisticAdventure #SpaceExploration\\n\\n### Post 2: Character Introduction\\n\\n**Image/Video:** A high-quality image of a main character in a dramatic pose.\\n\\n**Caption:**\\n"Meet [Character Name], the hero of \\\'Nexus Point\\\'. 🌟 Will they save the universe or face its greatest challenges? Find out when the series premieres! 🌌✨ #NexusPoint #CharacterReveal #SciFiHeroes 

## Benefits of This Simplified Routing Approach

- **Reduced Code**: Eliminated custom tools and complex response handling
- **Resource Efficiency**: Reuses the same model for classification and social media tasks
- **Simplified Classification**: Direct prompt engineering instead of structured output
- **Maintainability**: Fewer components and dependencies
- **Robustness**: Based on validated model compatibility and response handling